In [3]:
import pandas as pd  #pandas for using dataframe and reading csv 
import numpy as np   #numpy for vector operations and basic maths
import scipy as sp
from sklearn.model_selection import train_test_split
#import simplejson    #getting JSON in simplified format
import datetime as dt
import urllib        #for url stuff
#import gmaps       #for using google maps to visulalize places on maps
import re            #for processing regular expressions
import datetime      #for datetime operations
import calendar      #for calendar for datetime operations
import time          #to get the system time
import scipy         #for other dependancies
from sklearn.cluster import KMeans # for doing K-means clustering
from haversine import haversine # for calculating haversine distance
import math          #for basic maths operations
import seaborn as sns #for making plots
import matplotlib.pyplot as plt # for plotting
import os  # for os commands
from scipy.misc import imread, imresize, imsave  # for plots 
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from keras import utils as np_utils

/Users/vsanjeev/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Quick Run Script

##### Provides you with X & y to directly consume

In [4]:
Sel_Features_DecisionVar = pd.read_pickle('./full/Sel_Features_DecisionVar.pickle', compression='gzip')

In [5]:
exclude_features = [
    'pickup_datetime',
    'pickup_x', 'pickup_y',
    'dropoff_x', 'dropoff_y',
    'Month', 'Hour', 'DayOfWeek',
    'pickup_cluster', 'dropoff_cluster',
    'log_duration', 'duration'
]

In [6]:
features_with_dummies = list(set(Sel_Features_DecisionVar.columns) - set(exclude_features))

In [7]:
selected_features = features_with_dummies
decision_var = [
    'log_duration'
    #'duration'
]

In [8]:
X = Sel_Features_DecisionVar[selected_features]
y = Sel_Features_DecisionVar[decision_var]

##### Set seed

In [9]:
seed = 140
np.random.seed(seed)

##### Train - Test Split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Data Pre-Processing & Feature Engineering

In [ ]:
pd.options.display.max_rows = 250

In [ ]:
gotham_cabs_train = pd.read_csv('Train.csv')

In [ ]:
quantiles

In [ ]:
quantile = 0.05
quantiles = gotham_cabs_train.quantile([quantile, 1-quantile])
gotham_cabs_train_fil = gotham_cabs_train.copy()
gotham_cabs_train_fil = gotham_cabs_train_fil.loc[(gotham_cabs_train.pickup_x > quantiles.loc[quantile].pickup_x) & (gotham_cabs_train.pickup_x < quantiles.loc[1-quantile].pickup_x)]
gotham_cabs_train_fil = gotham_cabs_train_fil.loc[(gotham_cabs_train.pickup_y > quantiles.loc[quantile].pickup_y) & (gotham_cabs_train.pickup_y < quantiles.loc[1-quantile].pickup_y)]
gotham_cabs_train_fil = gotham_cabs_train_fil.loc[(gotham_cabs_train.dropoff_x > quantiles.loc[quantile].dropoff_x) & (gotham_cabs_train.dropoff_x < quantiles.loc[1-quantile].dropoff_x)]
gotham_cabs_train_fil = gotham_cabs_train_fil.loc[(gotham_cabs_train.dropoff_y > quantiles.loc[quantile].dropoff_y) & (gotham_cabs_train.dropoff_y < quantiles.loc[1-quantile].dropoff_y)]
gotham_cabs_train_fil = gotham_cabs_train_fil.loc[(gotham_cabs_train.duration > quantiles.loc[quantile].duration) & (gotham_cabs_train.duration < quantiles.loc[1-quantile].duration)]

In [ ]:
gotham_cabs_train_fil.head()

In [ ]:
gotham_cabs_train_fil['Month'] = gotham_cabs_train_fil['pickup_datetime'].apply(lambda s: dt.datetime.strptime(s, '%Y-%m-%d %H:%M:%S').month)
gotham_cabs_train_fil['Hour'] = gotham_cabs_train_fil['pickup_datetime'].apply(lambda s: dt.datetime.strptime(s, '%Y-%m-%d %H:%M:%S').hour)
gotham_cabs_train_fil['DayOfWeek'] = gotham_cabs_train_fil['pickup_datetime'].apply(lambda s: dt.datetime.strptime(s, '%Y-%m-%d %H:%M:%S').isoweekday())

In [ ]:
def minDistance(_record):
    return sp.spatial.distance.euclidean([_record['pickup_x'], _record['pickup_y']], [_record['dropoff_x'], _record['dropoff_y']])

In [ ]:
def manhattanDistance(_record):
    return sp.spatial.distance.cityblock([_record['pickup_x'], _record['pickup_y']], [_record['dropoff_x'], _record['dropoff_y']])

In [ ]:
def haversineDistance(_record):
    return haversine((_record['pickup_x'], _record['pickup_y']), (_record['dropoff_x'], _record['dropoff_y']))

In [ ]:
gotham_cabs_train_fil['MinDistance'] = gotham_cabs_train_fil.apply(lambda s: minDistance(s), axis=1)

In [ ]:
gotham_cabs_train_fil['HaversineDistance'] = gotham_cabs_train_fil.apply(lambda s: haversineDistance(s), axis=1)

In [ ]:
gotham_cabs_train_fil['ManhattanDistance'] = gotham_cabs_train_fil.apply(lambda s: manhattanDistance(s), axis=1)

In [ ]:
gotham_cabs_train_fil['log_duration'] = gotham_cabs_train_fil.apply(lambda s: 0 if s.duration==0 else np.log(s.duration), axis=1)

In [ ]:
gotham_cabs_train_fil.to_pickle('gotham_cabs_train_fil.pickle')

# 10% Random Sample for Analysis

In [ ]:
gotham_cabs_train_fil = pd.read_pickle('gotham_cabs_train_fil.pickle')

In [ ]:
sampled_gotham_city = gotham_cabs_train_fil.sample(frac=1, replace=False)

## K-Means

In [ ]:
sampled_gotham_city.columns

### Pick-Up Clustering

In [ ]:
#pickup_df = X[['pickup_x', 'pickup_y']]

In [ ]:
# kmeans = KMeans(n_clusters=60).fit(pickup_df)
# pickup_cluster = kmeans.predict(pickup_df)

In [ ]:
#X['pickup_cluster'] = pickup_cluster

In [ ]:
#plt.scatter(X['pickup_x'], X['pickup_y'], c=pickup_cluster, s=10)

### Drop-Up Clustering

In [ ]:
#dropoff_df = X[['dropoff_x', 'dropoff_y']]

In [ ]:
#kmeans = KMeans(n_clusters=60).fit(dropoff_df)
#dropoff_cluster = kmeans.predict(dropoff_df)

In [ ]:
#X['dropoff_cluster'] = dropoff_cluster

In [ ]:
#plt.scatter(X['dropoff_x'], X['dropoff_y'], c=dropoff_cluster, s=10)

### Pick-Drop Joint Clustering

In [ ]:
#x_coord = pd.concat([sampled_gotham_city['pickup_x'], sampled_gotham_city['dropoff_x']], axis=0)

In [ ]:
#y_coord = pd.concat([sampled_gotham_city['pickup_y'], sampled_gotham_city['dropoff_y']], axis=0)

In [ ]:
#xy_coord = pd.concat([x_coord, y_coord], axis=1)

In [ ]:
#kmeans = KMeans(n_clusters=600).fit(xy_coord)

In [ ]:
#from sklearn.externals import joblib
#joblib.dump(kmeans, 'kmeans_600.pkl') 

In [ ]:
kmeans = joblib.load('kmeans_600.pkl') 

In [ ]:
pickup_df = sampled_gotham_city[['pickup_x', 'pickup_y']]
pickup_cluster = kmeans.predict(pickup_df)
sampled_gotham_city['pickup_cluster'] = pickup_cluster

In [ ]:
dropoff_df = sampled_gotham_city[['dropoff_x', 'dropoff_y']]
dropoff_cluster = kmeans.predict(dropoff_df)
sampled_gotham_city['dropoff_cluster'] = dropoff_cluster

In [ ]:
pickup_cluster = kmeans.predict(xy_coord)
plt.scatter(xy_coord[0], xy_coord[1], c=pickup_cluster, s=10)

In [ ]:
sampled_gotham_city.to_pickle('sampled_gotham_city.pickle')

## Into-Categorical

In [ ]:
sampled_gotham_city = pd.read_pickle('sampled_gotham_city.pickle')

In [ ]:
sampled_gotham_city.columns

In [ ]:
# sampled_gotham_city['Month'] = np_utils.to_categorical(sampled_gotham_city['Month'])
# sampled_gotham_city['Hour'] = np_utils.to_categorical(sampled_gotham_city['Hour'])
# sampled_gotham_city['DayOfWeek'] = np_utils.to_categorical(sampled_gotham_city['DayOfWeek'])

# sampled_gotham_city['pickup_cluster'] = np_utils.to_categorical(sampled_gotham_city['pickup_cluster'])
# sampled_gotham_city['dropoff_cluster'] = np_utils.to_categorical(sampled_gotham_city['dropoff_cluster'])

In [ ]:
df_with_dummies = pd.get_dummies(sampled_gotham_city, columns = ['Month', 'Hour', 'DayOfWeek', 'pickup_cluster', 'dropoff_cluster'])

In [ ]:
df_with_dummies.to_pickle('sampled_gotham_city_cat.pickle')

In [ ]:
#sampled_gotham_city.to_pickle('sampled_gotham_city_cat.pickle')

## Feature Selection

In [4]:
sampled_gotham_city_cat = pd.read_pickle('./full/sampled_gotham_city_cat.pickle')

In [1]:
exclude_features = [
    'pickup_datetime',
    'pickup_x', 'pickup_y',
    'dropoff_x', 'dropoff_y',
    'Month', 'Hour', 'DayOfWeek',
    'pickup_cluster', 'dropoff_cluster',
    'log_duration', 'duration'
]

In [5]:
features_with_dummies = list(set(sampled_gotham_city_cat.columns) - set(exclude_features))

In [ ]:
features_with_dummies

In [6]:
# selected_features = [
#     #'pickup_datetime',
#     #'pickup_x', 'pickup_y',
#     #'dropoff_x', 'dropoff_y',
#     'Month', 'Hour', 'DayOfWeek',
#     'MinDistance',
#     'HaversineDistance', 
#     'ManhattanDistance',
#     'pickup_cluster', 'dropoff_cluster'
# ]
selected_features = features_with_dummies
decision_var = [
    'log_duration'
    #'duration'
]

In [ ]:
df = sampled_gotham_city_cat.copy(deep=True)

In [ ]:
X = df[selected_features]
y = df[decision_var]

### Normalization of Selected Variables

In [ ]:
Sel_Features_DecisionVar = pd.concat([X,y], axis=1)

In [ ]:
missing_df = Sel_Features_DecisionVar.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
print(scaler.fit(Sel_Features_DecisionVar))

In [ ]:
Sel_Features_DecisionVar[list(Sel_Features_DecisionVar.columns)] = scaler.transform(Sel_Features_DecisionVar)

In [ ]:
Sel_Features_DecisionVar

In [ ]:
#Sel_Features_DecisionVar.to_pickle('Sel_Features_DecisionVar.pickle', compression='gzip')

In [7]:
Sel_Features_DecisionVar = pd.read_pickle('./full/Sel_Features_DecisionVar.pickle', compression='gzip')

In [8]:
X = Sel_Features_DecisionVar[selected_features]
y = Sel_Features_DecisionVar[decision_var]

In [9]:
print(X.shape, y.shape)

(876761, 1209) (876761, 1)


### Test - Train Split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [11]:
X_train.head()

,dropoff_cluster_526,pickup_cluster_197,dropoff_cluster_94,dropoff_cluster_463,pickup_cluster_269,pickup_cluster_5,dropoff_cluster_351,dropoff_cluster_428,pickup_cluster_44,pickup_cluster_154,...,pickup_cluster_533,pickup_cluster_38,dropoff_cluster_338,dropoff_cluster_501,dropoff_cluster_114,dropoff_cluster_339,pickup_cluster_19,pickup_cluster_33,pickup_cluster_322,dropoff_cluster_273
260886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
170798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
696300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1233736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
y_train.head()

,log_duration
260886,0.351030
36879,0.846971
170798,0.218535
696300,0.693249
1233736,0.860632


## Neural Network

In [11]:
import numpy
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

In [12]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
# def baseline_model():
#     # create model
#     model = Sequential()
#     model.add(Dense(1205, input_dim=1209, kernel_initializer='normal', activation='relu'))
#     model.add(Dense(1, kernel_initializer='normal'))
#     # Compile model
#     model.compile(loss='mean_squared_error', optimizer='adam')
#     return model

In [ ]:
# estimator = KerasRegressor(build_fn=baseline_model, epochs=2, batch_size=1000, verbose=0)

In [15]:
seed = 140
np.random.seed(seed)

In [ ]:
# kfold = KFold(n_splits=10, random_state=seed)
# results = cross_val_score(estimator, X, y, cv=kfold)
# print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [13]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(1205, input_dim=1209, activation='sigmoid'))
    #model.add(Conv2D(20, (7, 7), input_shape=(10), activation='relu'))
    #model.add(MaxPooling2D(pool_size=(4, 4)))
    #model.add(Dropout(0.25))
    #model.add(Flatten())
    model.add(Dense(10000, activation='sigmoid'))
    model.add(Dense(1296, activation='hard_sigmoid'))
    model.add(Dense(36, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae', 'mape', 'cosine'])
    return model

In [14]:
# build the model
model = larger_model()

In [ ]:
# Fit the model: For more accurate results increase epochs to 10
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=2000)
# Final evaluation of the model

In [ ]:
#model.save('nn_model.h5')

In [15]:
estimator = KerasRegressor(build_fn=larger_model, epochs=3, batch_size=2000, verbose=1)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, y, cv=kfold, n_jobs=3)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/3
Epoch 1/3
 32000/789085 [>.............................] - ETA: 2:11:07 - loss: 0.2088 - mean_squared_error: 0.2088 - mean_absolute_error: 0.3985 - mean_absolute_percentage_error: 945519.3145 - cosine_proximity: -0.9989

KeyboardInterrupt: 

In [ ]:
# # evaluate model with standardized dataset
# np.random.seed(seed)
# estimators = []
# estimators.append(('standardize', StandardScaler()))
# estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=3, batch_size=2000, verbose=0)))
# pipeline = Pipeline(estimators)
# kfold = KFold(n_splits=10, random_state=seed)
# results = cross_val_score(pipeline, X, y, cv=kfold, n_jobs=3)
# print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

## Random Forest Regressor

In [ ]:
regr = RandomForestRegressor(max_depth=6)
regr.fit(X_train, y_train.values.ravel())

In [ ]:
regr.feature_importances_

In [ ]:
y_pred = regr.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred)

In [ ]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(regr, X, y.values.ravel(), cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))